In [1]:
import pandas as pd
import subprocess
import ast
import re

## Function to extract the outcome

In [23]:
def extract_case_outcome(text, start, end, model="llama3"):
    """
    Extract the outcome of a legal case in a single word.

    Parameters
    ----------
    text : str
        The court text from which to extract the outcome.
    model : str, optional
        The language model to use (default is "llama3").

    Returns
    -------
    str
        A single word summarizing the case outcome (e.g., "allowed", "dismissed", "granted").
        Returns "unknown" if no outcome is identified.
    """
    lines = text.splitlines()
    relevant_text = "\n".join(lines[start:end])
    prompt = f"""
You are a legal assistant. Your task is to determine the outcome of the court case based on the provided excerpt.

Instructions:
- Read the text carefully and identify the final decision or outcome.
- Return only one lowercase word that best summarizes the outcome (e.g., "allowed", "dismissed", etc).
- If the outcome is unclear or not mentioned, return "unknown".

Court Text:
{relevant_text}

Output:
"""

    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode(),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    return result.stdout.decode().strip().lower()

def extract_case_outcomes_from_dataframe(df, start, end, text_column="unofficial_text", model="llama3"):
    """
    Extract the case outcome (single word) from a DataFrame containing court texts.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing court text data.
    text_column : str, optional
        The column name in `df` that contains the court text (default is "unofficial_text").
    model : str, optional
        The language model to use (default is "llama3").

    Returns
    -------
    pandas.DataFrame
        A new DataFrame with an additional 'outcome' column containing the extracted outcome word.
    """
    df = df.copy()
    df["outcome"] = None

    for idx, row in df.iterrows():
        full_text = row[text_column]
        try:
            outcome = extract_case_outcome(full_text, start, end, model=model)
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            outcome = "unknown"

        df.at[idx, "outcome"] = outcome
        print(f"Row {idx} processed with outcome: {outcome}")
    
    return df

## Federal Court

In [ ]:
FC = pd.read_csv("../data/processed/FC_city.csv")
FC

In [ ]:
FC_outcome = extract_case_outcomes_from_dataframe(FC, -50, -20)
FC_outcome

In [ ]:
FC_final = FC_outcome.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
FC_final

In [ ]:
FC_final['outcome'].value_counts()

In [ ]:
FC_final.to_excel("../data/processed/inadmissibility_court_cases.xlsx")